In [19]:
import numpy as np
import pandas as pd

In [20]:
df_train = pd.read_csv('quora-question-pairs/train.csv')
df_train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [21]:
df_train.shape

(404290, 6)

In [22]:
def check_null_rows(df):
    is_NaN = df.isnull()
    row_has_NaN = is_NaN.any(axis=1)
    rows_with_NaN = df[row_has_NaN]
    return rows_with_NaN

In [23]:
check_null_rows(df_train)

,id,qid1,qid2,question1,question2,is_duplicate
105780,105780,174363,174364,How can I develop android app?,NaN,0
201841,201841,303951,174364,How can I create an Android app?,NaN,0
363362,363362,493340,493341,NaN,My Chinese name is Haichao Yu. What English na...,0


In [24]:
df_train = df_train.sample(30000, random_state=42)

In [25]:
df_train.shape

(30000, 6)

In [26]:
check_null_rows(df_train)

,id,qid1,qid2,question1,question2,is_duplicate


# Feature Engineering

### Column's with number of character in each question

In [27]:
df_train['q1_char_len'] = df_train['question1'].str.len()
df_train['q2_char_len'] = df_train['question2'].str.len()

In [28]:
df_train.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_char_len,q2_char_len
8067,8067,15738,15739,How do I play Pokémon GO in Korea?,How do I play Pokémon GO in China?,0,34,34
368101,368101,12736,104117,What are some of the best side dishes for crab...,What are some good side dishes for buffalo chi...,0,53,51
70497,70497,121486,121487,Which is more advisable and better material fo...,What is the best server setup for buddypress?,0,96,45
226567,226567,254474,258192,How do I improve logical programming skills?,How can I improve my logical skills for progra...,1,44,52
73186,73186,48103,3062,How close we are to see 3rd world war?,How close is a World War III?,1,38,29


### Column's with number of words in each question

In [29]:
df_train['q1_word_len'] = df_train['question1'].apply(lambda x: len(x.split(" ")))
df_train['q2_word_len'] = df_train['question2'].apply(lambda x: len(x.split(" ")))

In [30]:
df_train.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_char_len,q2_char_len,q1_word_len,q2_word_len
8067,8067,15738,15739,How do I play Pokémon GO in Korea?,How do I play Pokémon GO in China?,0,34,34,8,8
368101,368101,12736,104117,What are some of the best side dishes for crab...,What are some good side dishes for buffalo chi...,0,53,51,11,9
70497,70497,121486,121487,Which is more advisable and better material fo...,What is the best server setup for buddypress?,0,96,45,16,8
226567,226567,254474,258192,How do I improve logical programming skills?,How can I improve my logical skills for progra...,1,44,52,7,9
73186,73186,48103,3062,How close we are to see 3rd world war?,How close is a World War III?,1,38,29,9,7


### Column with total common words in question1 and question2

In [72]:
def common_words(row):
    word_list1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
    word_list2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))
    return len(word_list1 & word_list2)

In [74]:
 df_train['common_words'] = df_train.apply(common_words, axis=1)

In [75]:
df_train.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_char_len,q2_char_len,q1_word_len,q2_word_len,common_words
8067,8067,15738,15739,How do I play Pokémon GO in Korea?,How do I play Pokémon GO in China?,0,34,34,8,8,7
368101,368101,12736,104117,What are some of the best side dishes for crab...,What are some good side dishes for buffalo chi...,0,53,51,11,9,6
70497,70497,121486,121487,Which is more advisable and better material fo...,What is the best server setup for buddypress?,0,96,45,16,8,2
226567,226567,254474,258192,How do I improve logical programming skills?,How can I improve my logical skills for progra...,1,44,52,7,9,4
73186,73186,48103,3062,How close we are to see 3rd world war?,How close is a World War III?,1,38,29,9,7,3


### Column's with percentage of common words in questions